# Modell

## Komponenten

```{figure} ./images/model_aufbau.png
:name: model_aufbau
:align: center
Komponenten des Segment Anything Models. Image Encoder, Prompt Encoder, and Mask Decoder. {cite}`kirillov2023segment`
```

### 1. Image Encoder
- Erstellt **Image Embeddings** für Input-Bilder
- In Theorie kann jede Art von Image Encoder verwendet werden
    - Vorausgesetzt Output ist ein (C×H×W) Embedding
- **Vision Transformer** (ViT) von {site} `dosovitskiy2021image`. Angepasst für hochauflösende Input Bilder
    - pre-trained mit **Masked Autoencoder** (MAE) Verfahren
    - ViT-H/16
    - 14x14 windowed attention Blocks
    - 4 global attention Blocks

```{figure} ./images/ImageEncoderDiagram.png
:name: ImageEncoderDiagram
:align: center
SAM Image Decoder Struktur im Detail. {cite}`kirillov2023segment`
```

### 2. Prompt Encoder
- Prompts werden je nach Art unterschiedlich encoded

| Type   | Prompt | Embedding                                  |
|--------|--------|--------------------------------------------|
| Sparse | Points | Positional Encoding + gelernte Embeddings |
| Sparse | Boxes  | Positional Encoding + gelernte Embeddings |
| Sparse | Text   | CLIP Encoder                               |
| Dense  | Mask   | Convolution Embedding + Image Embedding    |

- Sparse Prompts werden auf ein **256-dimensional vectorial Embedding** gemappt:
    - **Points:** Positional Encodings der Koordinaten summiert mit trainierten Embeddings für Forder- bzw. Hintergrund.
    - **Box:** Embedding Paar: Positional Encoding von Koordinaten "Oben Links" und "Unten Rechts" werden mit zugehörigen gelernten Embeddings summiert.
    - **Text:** CLIP Encoder. Jedoch jeder Text Encoder theoretisch möglich.
- **Masks** (Dense Prompts) werden gedownscaled und durch mehrere Convolution Layers transformiert (Siehe Abbildung xx).
 
```{figure} ./images/MaskPromptEncoding.png
:name: MaskPromptEncoder
:align: center
SAM Mask Prompt Encoding Struktur. {cite}`kirillov2023segment`
```


### 3. Mask Decoder
- Mappt Image Encoding, Prompt Encoding und ein Output Token auf eine Mask
    - Vor Decoding wird dem Prompt Embedding ein trainiertes **Output Token Embedding** hinzugefügt.
- Modifizierter **Transformer Decoder Block** gefolgt von einem **Dynamic Mask Prediction Head**
 
- Inspiriert von Transformer Architekturen von {site} `carion2020endtoend` und {site} `cheng2021perpixel`

```{figure} ./images/mask_decoder_model.png
:name: mask_decoder_model
:align: center
SAM Mask Decoder. {cite}`kirillov2023segment`
```


## Training

- Nutzung eines **Interactive Segmentation Setup** angelehnt an {cite}`sofiiuk2021reviving` und {cite}`forte2020getting`.
- 11 Interationen pro Trainingsschritt:
    - Erste Prediction mit Bounding Box oder Point Prompt
    - 8 Iterative Predictions mit Vorheriger Output Maske und gesampelten Punkten der Prediction Error Region
    - Zwei zusätzliche Iterationen ohne zusätzliche Punkte (Nur vorherige Output Maske)
    
- Durch die gute Performance des Decoders können deutlich mehr Iterationen pro Trainingsschritt als in vorherigen Intractive Segmentation Projekten verwendet werden.